<h1>KMeans</h1>

## Importación de librerias



In [40]:

import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, silhouette_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import mode  




# Dataset

## Loading data

In [2]:
df_train = pd.read_parquet('../data/processed/df_train_reduced.parquet')
df_test= pd.read_parquet('../data/processed/df_test_reduced.parquet')
df_test_missing_10 = pd.read_parquet('../data/processed/df_test_reduced_missing_10.parquet')
df_test_missing_20 = pd.read_parquet('../data/processed/df_test_reduced_missing_20.parquet')
df_test_missing_30 = pd.read_parquet('../data/processed/df_test_reduced_missing_30.parquet')
df_test_missing_40 = pd.read_parquet('../data/processed/df_test_reduced_missing_40.parquet')

In [3]:
df_test_missing_10.head(10)

,10335183_chrXV_303214_G_T,10341838_chrXV_309869_T_C,10341923_chrXV_309954_G_A,10342190_chrXV_310221_T_C,10342543_chrXV_310574_A_G,10344120_chrXV_312151_C_T,10346156_chrXV_314187_C_G,10349298_chrXV_317329_C_T,3141305_chrV_249350_C_T,10383039_chrXV_351070_C_T,...,10640336_chrXV_608367_C_T,1428699_chrIV_68677_T_C,1428933_chrIV_68911_A_C,6970183_chrXI_391971_C_T,6970060_chrXI_391848_C_T,6969970_chrXI_391758_T_A,3698349_chrVI_229520_A_G,6969924_chrXI_391712_C_T,5535875_chrIX_143302_C_T,7948843_chrXII_703815_C_T
0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,2.0,...,2.0,1.0,1.0,1.0,1.0,NaN,2.0,1.0,1.0,1.0
1,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0
2,2.0,2.0,NaN,2.0,NaN,NaN,2.0,2.0,1.0,2.0,...,2.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,2.0,1.0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,...,NaN,2.0,2.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0
4,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,1.0,1.0,2.0,2.0,2.0,2.0,NaN,1.0,1.0
5,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.0,1.0,1.0,2.0,2.0,NaN,1.0,2.0,2.0,2.0
6,NaN,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,NaN,...,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0
7,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,...,2.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,2.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,...,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN
9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.0,2.0,NaN,2.0,2.0,2.0,1.0,NaN,2.0,1.0


In [4]:
df_train.head(10)

,10335183_chrXV_303214_G_T,10341838_chrXV_309869_T_C,10341923_chrXV_309954_G_A,10342190_chrXV_310221_T_C,10342543_chrXV_310574_A_G,10344120_chrXV_312151_C_T,10346156_chrXV_314187_C_G,10349298_chrXV_317329_C_T,3141305_chrV_249350_C_T,10383039_chrXV_351070_C_T,...,10640336_chrXV_608367_C_T,1428699_chrIV_68677_T_C,1428933_chrIV_68911_A_C,6970183_chrXI_391971_C_T,6970060_chrXI_391848_C_T,6969970_chrXI_391758_T_A,3698349_chrVI_229520_A_G,6969924_chrXI_391712_C_T,5535875_chrIX_143302_C_T,7948843_chrXII_703815_C_T
0,1,1,1,1,1,1,1,1,2,2,...,2,2,2,1,1,1,2,1,1,1
1,2,2,2,2,2,2,2,2,1,1,...,1,2,2,1,1,1,1,1,1,2
2,2,2,2,2,2,2,2,1,2,1,...,1,1,1,2,2,2,1,2,2,1
3,1,1,1,1,1,1,1,1,1,2,...,2,1,1,2,2,2,2,2,2,2
4,2,2,2,2,2,2,2,2,1,2,...,1,2,2,1,1,1,2,1,2,1
5,2,2,2,2,2,2,2,2,2,2,...,2,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,1,2,1,2
7,2,2,2,2,2,2,2,2,1,1,...,1,2,2,2,2,2,2,2,1,1
8,1,1,1,1,1,1,1,1,2,1,...,1,1,1,1,1,1,1,1,2,1
9,1,1,1,1,1,1,1,1,2,2,...,2,2,2,1,1,1,2,1,1,2


### Mostramos el numero de clases de entrenamieno y verificamos que están balanceadas

In [5]:
# Contar los valores de 0 y 1 en todo el DataFrame
total_counts = df_train.apply(pd.Series.value_counts).sum(axis=1)
print("Conteo total de clases en todo el DataFrame:")
print(total_counts)

Conteo total de clases en todo el DataFrame:
1    1756511
2    1756489
dtype: int64


## Modelo: KMeans

Se define la función que se encarga de entrenar el modelo

In [9]:

# Entrenamiento con K-Means
def train_model(df_train, n_clusters):
    model = KMeans(n_clusters=n_clusters, random_state=42)
    model.fit(df_train)
    return model

### Test 

In [45]:


def predict_missing_values_for_row(row, model, known_features, missing_features):
    X_known = row[known_features].values.reshape(1, -1)

    # Crear un dataframe vacío con las columnas del modelo y rellenar con ceros
    X_full = pd.DataFrame(np.zeros((1, model.n_features_in_)), columns=model.feature_names_in_)
    X_full.loc[:, known_features] = X_known

    # Asignar el cluster a la fila basada en las características conocidas
    cluster = model.predict(X_full)[0]

    # Obtener los índices de las características faltantes
    missing_indices = [list(model.feature_names_in_).index(feat) for feat in missing_features]

    # Usar el centroide del cluster para imputar los valores faltantes
    pred = model.cluster_centers_[cluster][missing_indices]
    return pred, missing_features

# Imputar valores faltantes usando clustering
def impute_and_evaluate_clustering(df_test_missing, df_test_original, model):
    df_test_imputed = df_test_missing.copy()

    all_y_true = []
    all_y_pred = []

    for index, row in df_test_missing.iterrows():
        if row.isna().any():  # Solo procesar filas con valores faltantes
            known_features = row.dropna().index  # Características conocidas
            missing_features = row.index[row.isna()]  # Características faltantes

            if len(known_features) > 0:
                # Predecir los valores faltantes
                pred, missing_features = predict_missing_values_for_row(row, model, known_features, missing_features)
                
                # Imputar los valores predichos en el dataset
                df_test_imputed.loc[index, missing_features] = pred

                # Guardar los valores verdaderos y predichos para evaluación
                y_true = df_test_original.loc[index, missing_features]
                if isinstance(y_true, pd.Series):
                    if len(y_true) == len(pred):
                        all_y_true.extend(y_true.values)  # Convertir a lista de valores escalares
                        all_y_pred.extend(pred)
                else:
                    all_y_true.append(y_true)
                    all_y_pred.append(pred)

    # Evaluar la imputación con varias métricas
    mse = mean_squared_error(all_y_true, all_y_pred)
    mae = mean_absolute_error(all_y_true, all_y_pred)
    r2 = r2_score(all_y_true, all_y_pred)

    return df_test_imputed, mse, mae, r2

# Proceso para imputar diferentes niveles de missingness usando clustering
def process_clustering(df_train, df_test, missingness_levels,model, n_clusters=2):
    results = {
        "MSE": [],
        "MAE": [],
        "R2": []
    }


    for level in missingness_levels:
        df_test_missing = globals()[f'df_test_missing_{level}']  # Conjunto de test con missingness
        
        # Imputar valores faltantes y evaluar
        df_test_imputed, mse, mae, r2 = impute_and_evaluate_clustering(df_test_missing, df_test, model)

        # Guardar las métricas
        results["MSE"].append(mse)
        results["MAE"].append(mae)
        results["R2"].append(r2)

        # Guardar el dataset imputado
        globals()[f'df_test_missing_{level}_imputed'] = df_test_imputed

    return results

# Función para reportar las métricas obtenidas
def detailed_metrics_report(missingness_levels, metrics_dict):
    print("==== Reporte Detallado de Métricas ====\n")
    for metric_name, metric_values in metrics_dict.items():
        print(f"Métricas para {metric_name}:")
        for i, level in enumerate(missingness_levels):
            print(f"- Porcentaje de valores faltantes {level}%: {metric_values[i]:.4f}")
        print("\n")
    


In [64]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

# Paso 1: Entrenar el modelo KMeans para encontrar dos clusters
# Función para entrenar KMeans y asignar etiquetas a los clusters
def train_kmeans_for_classification(df_train, n_clusters=2):
    model = KMeans(n_clusters=n_clusters, random_state=42)
    model.fit(df_train)
    
    cluster_labels = {}
    for cluster in range(n_clusters):
        # Obtener los datos pertenecientes a este cluster
        cluster_data = df_train[model.labels_ == cluster]
        
        if len(cluster_data) > 0:
            # Calcular la moda de los valores en el cluster
            cluster_values = cluster_data.values.ravel()
            mode_result = mode(cluster_values)
            
            # Extraer la moda directamente
            if mode_result.count[0] > 0:
                most_common_value = mode_result.mode[0]
            else:
                most_common_value = np.nan
            
            cluster_labels[cluster] = most_common_value
        else:
            # Si no hay datos, asignar una moda global o valor por defecto
            most_common_value = mode(df_train.values.ravel()).mode[0]
            cluster_labels[cluster] = most_common_value
    
    return model, cluster_labels

def predict_missing_values_for_row(row, model, cluster_labels):
    # Predecir el cluster al que pertenece la fila
    cluster = model.predict(row.dropna().values.reshape(1, -1))[0]
    
    # Asignar la etiqueta del cluster
    predicted_label = cluster_labels.get(cluster, np.nan)
    
    return predicted_label

def impute_with_classification(df_test_missing, model, cluster_labels):
    df_test_imputed = df_test_missing.copy()
    
    for index, row in df_test_missing.iterrows():
        if row.isna().any():
            # Imputar el valor predicho
            predicted_label = predict_missing_values_for_row(row, model, cluster_labels)
            missing_features = row.index[row.isna()]
            
            # Imputar el valor predicho en las características faltantes
            df_test_imputed.loc[index, missing_features] = predicted_label
    
    return df_test_imputed



from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_imputation(df_test_missing, df_test, model, cluster_labels):
    # Imputar los valores faltantes
    df_test_imputed = impute_with_classification(df_test_missing, model, cluster_labels)
    
    # Calcular las métricas de evaluación
    mse = mean_squared_error(df_test.fillna(0), df_test_imputed.fillna(0))
    mae = mean_absolute_error(df_test.fillna(0), df_test_imputed.fillna(0))
    r2 = r2_score(df_test.fillna(0), df_test_imputed.fillna(0))
    
    return df_test_imputed, mse, mae, r2

def process_clustering(df_train, df_test, missingness_levels, n_clusters=2):
    results = {
        "MSE": [],
        "MAE": [],
        "R2": []
    }
    
    # Entrenar el modelo KMeans
    kmeans_model, cluster_labels = train_kmeans_for_classification(df_train, n_clusters=n_clusters)
    
    for level in missingness_levels:
        # Crear un conjunto de datos con el porcentaje de valores faltantes
        df_test_missing = df_test.copy()
        mask = np.random.rand(*df_test.shape) < level / 100
        df_test_missing[mask] = np.nan
        
        # Imputar y evaluar
        _, mse, mae, r2 = evaluate_imputation(df_test_missing, df_test, kmeans_model, cluster_labels)
        
        # Guardar las métricas
        results["MSE"].append(mse)
        results["MAE"].append(mae)
        results["R2"].append(r2)
    
    return results


In [11]:
KMeans_model = train_model(df_train,2)  # Aquí entrenamos el modelo una sola vez


In [65]:
missingness_levels = [10, 20, 30, 40]  # Niveles de valores faltantes
results = process_clustering(df_train, df_test, missingness_levels,n_clusters=2)

IndexError: invalid index to scalar variable.

In [48]:
# Paso 3: Mostrar los resultados obtenidos
detailed_metrics_report(missingness_levels, results)


==== Reporte Detallado de Métricas ====

Métricas para MSE:
- Porcentaje de valores faltantes 10%: 0.2330
- Porcentaje de valores faltantes 20%: 0.2481
- Porcentaje de valores faltantes 30%: 0.2673
- Porcentaje de valores faltantes 40%: 0.2677


Métricas para MAE:
- Porcentaje de valores faltantes 10%: 0.4650
- Porcentaje de valores faltantes 20%: 0.4800
- Porcentaje de valores faltantes 30%: 0.4992
- Porcentaje de valores faltantes 40%: 0.4997


Métricas para R2:
- Porcentaje de valores faltantes 10%: 0.0679
- Porcentaje de valores faltantes 20%: 0.0078
- Porcentaje de valores faltantes 30%: -0.0690
- Porcentaje de valores faltantes 40%: -0.0709




In [49]:
df_test_missing_10_imputed

,10335183_chrXV_303214_G_T,10341838_chrXV_309869_T_C,10341923_chrXV_309954_G_A,10342190_chrXV_310221_T_C,10342543_chrXV_310574_A_G,10344120_chrXV_312151_C_T,10346156_chrXV_314187_C_G,10349298_chrXV_317329_C_T,3141305_chrV_249350_C_T,10383039_chrXV_351070_C_T,...,10640336_chrXV_608367_C_T,1428699_chrIV_68677_T_C,1428933_chrIV_68911_A_C,6970183_chrXI_391971_C_T,6970060_chrXI_391848_C_T,6969970_chrXI_391758_T_A,3698349_chrVI_229520_A_G,6969924_chrXI_391712_C_T,5535875_chrIX_143302_C_T,7948843_chrXII_703815_C_T
0,1.00000,1.445969,1.000000,1.0,1.000000,1.000000,1.447113,1.0,1.000000,2.0,...,2.000000,1.000000,1.000000,1.000000,1.0,1.500858,2.000000,1.000000,1.000000,1.0
1,1.00000,1.000000,1.445969,1.0,1.000000,1.000000,1.000000,1.0,2.000000,1.0,...,1.000000,2.000000,2.000000,2.000000,2.0,2.000000,1.000000,2.000000,1.000000,2.0
2,2.00000,2.000000,1.553855,2.0,1.553855,1.553288,2.000000,2.0,1.000000,2.0,...,2.000000,1.497166,1.000000,1.000000,1.0,1.499433,1.000000,1.000000,2.000000,1.0
3,2.00000,2.000000,2.000000,2.0,2.000000,2.000000,2.000000,2.0,2.000000,1.0,...,1.515873,2.000000,2.000000,1.499433,2.0,2.000000,2.000000,2.000000,1.516440,2.0
4,1.44311,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.0,...,1.483705,1.000000,1.000000,2.000000,2.0,2.000000,2.000000,1.500858,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,2.00000,2.000000,2.000000,2.0,2.000000,2.000000,2.000000,2.0,1.000000,2.0,...,1.000000,1.000000,1.502573,1.500858,1.0,1.000000,1.000000,1.000000,1.000000,1.0
873,1.00000,1.000000,1.000000,1.0,1.000000,1.446541,1.000000,1.0,1.000000,2.0,...,1.483705,1.000000,1.000000,1.000000,1.0,1.000000,2.000000,1.000000,1.000000,2.0
874,2.00000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.488851,1.0,...,1.483705,1.000000,1.000000,1.000000,1.0,1.000000,2.000000,1.000000,1.483705,1.0
875,2.00000,2.000000,2.000000,2.0,2.000000,2.000000,2.000000,2.0,2.000000,2.0,...,1.000000,1.000000,1.000000,2.000000,2.0,2.000000,1.495712,2.000000,2.000000,2.0


In [ ]:
# Paso 4: (Opcional) Graficar los resultados
plot_multiple_metrics(missingness_levels, results)

### Guardar datasets imputados a partir de los distintos datasets con diferentes porcentajes de missingness y modelo generado

In [25]:


def save_model(rf_model, model_path):
    os.makedirs(os.path.dirname(model_path), exist_ok=True)  # Crear directorios si no existen
    
    # Serializamos el modelo con pickle en un buffer de bytes
    model_bytes = pickle.dumps(rf_model)
    
    # Convertir el buffer a un array NumPy para que HDF5 lo maneje correctamente
    model_array = np.frombuffer(model_bytes, dtype='uint8')
    
    # Guardar el modelo en un archivo HDF5 con compresión GZIP
    with h5py.File(model_path, 'w') as f:
        # Dividimos el array en chunks y aplicamos compresión
        f.create_dataset('random_forest_model', data=model_array, compression="gzip", compression_opts=9)
    
    print(f"Modelo guardado y comprimido en {model_path}")


# Función para guardar los datasets imputados
def save_imputed_datasets(df_imputed, level, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # Crear directorios si no existen
    file_path = os.path.join(output_dir, f'df_test_missing_{level}_imputed.parquet')
    df_imputed.to_csv(file_path, index=False)
    print(f"Dataset imputado guardado en {file_path}")


In [26]:


# Guardar el modelo
model_output_path = "../models/XGBoost/XGBoost_model.h5"
save_model(xgb_model, model_output_path)

# Guardar los datasets imputados
for level in missingness_levels:
    df_test_imputed = globals()[f'df_test_missing_{level}_imputed']
    data_output_dir = "../data/generated/xgboost"
    save_imputed_datasets(df_test_imputed, level, data_output_dir)


Modelo guardado y comprimido en ../models/XGBoost/XGBoost_model.h5
Dataset imputado guardado en ../data/generated/xgboost/df_test_missing_10_imputed.parquet
Dataset imputado guardado en ../data/generated/xgboost/df_test_missing_20_imputed.parquet
Dataset imputado guardado en ../data/generated/xgboost/df_test_missing_30_imputed.parquet
Dataset imputado guardado en ../data/generated/xgboost/df_test_missing_40_imputed.parquet


Así se cargaría el modelo de ser necesario desde HDF5:
with h5py.File(model_output_path, 'r') as f:
    model_bytes = f['xgboost_model'][()]
    f_model_loaded = joblib.loads(model_bytes)
    